In [ ]:
from glob import glob
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
train_csv_paths = list(glob(os.path.join('.', 'train_*.csv')))
valid_csv_paths = list(glob(os.path.join('.', 'valid_*.csv')))
assert len(train_csv_paths) > 0

In [ ]:
df = pd.read_csv(train_csv_paths[0])

In [ ]:
new_columns = {col: col.replace('validation__', '') for col in df.columns if col.startswith('validation__')}
df = df.rename(columns=new_columns)

In [ ]:
unweighted_sample_cols = [
    str(col) for col in df.columns if 'unweigted_sample_loss' in col
]

In [ ]:
epochs = sorted(list(set(df.epoch_id)))
epochs

In [ ]:
unweighted_losses = []
for i in epochs:
    loss = df.loc[df.epoch_id == i].iloc[:, df.columns.isin(unweighted_sample_cols)].sum(axis=1).mean()
    unweighted_losses.append(loss)
plt.plot(unweighted_losses)

In [ ]:
cols = list(df.columns)
rotation_cols = list(col for col in cols if '_rotation_' in col and 'sample_loss' in col)
position_cols = list(col for col in cols if '_position_' in col and 'sample_loss' in col)
fig = plt.figure()
mean_loss = df[rotation_cols + position_cols + ['epoch_id']].groupby('epoch_id').mean()
ax = mean_loss.plot(ylabel='cross_entropy_loss')
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
per_task_loss = df[rotation_cols + position_cols + ['task']].groupby('task').sum().sum(axis=1).sort_values()
per_task_loss / per_task_loss.min()

Learning Rate

In [ ]:
import math 
from playground.typing import TrainParam, DatasetParam

def get_lr_param():
    return {
        "warmup_end_at_iters": 7000,
        "flatten_end_at_iters": 240000,
        "lr_decay_end_at_iters": 960000,
        "learning_rate": 1e-5,
        "min_lr": 1e-7, 
    }


def get_optimizer_param():
    return {
        "clip_norm": 1.0,
        "inital_lr": get_lr_param()["learning_rate"],
        "optimizer_name": "AdamW",
        "weight_decay": 0.0
    }

def get_dataset_param():
    return  {
        "data_pct_usage": 1.0,
        "total_data_size_per_task": 40000,
        "validation_pct": 0.00,
        "source": "s3://vima",
        "tasks": [
            "follow_order",
            "manipulate_old_neighbor",
            "novel_adj",
            "novel_noun",
            "pick_in_order_then_restore",
            "rearrange_then_restore",
            "rearrange",
            "rotate",
            "same_profile",
            "scene_understanding",
            "simple_manipulation",
            "sweep_without_exceeding",
            "twist",
        ]
    }

def get_train_param():
    return {
        "model_size": "2M",
        "total_epoch": 50,
        "local_batch_size": 128,
        "distributed": False,
    }
def get_lr(it: int) -> float:
    lr_param = get_lr_param()
    warmup_iters = lr_param["warmup_end_at_iters"]
    flatten_iters = lr_param["flatten_end_at_iters"]
    learning_rate = lr_param["learning_rate"]
    lr_decay_iters = lr_param["lr_decay_end_at_iters"]
    min_lr = lr_param["min_lr"]
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    if warmup_iters <= it < flatten_iters:
        return learning_rate
    if it > lr_decay_iters:
        return min_lr
    decay_ratio = (it - flatten_iters) / (lr_decay_iters - flatten_iters)
    assert 0 <= decay_ratio <= 1, f"{decay_ratio = }, {it = }"
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return min_lr + coeff * (learning_rate - min_lr)


def get_batch_per_epoch(
        dataset_param: DatasetParam,
        train_param: TrainParam,
        is_train: bool = True
    ):
    if is_train:
        scaling = 1.0
    else:
        scaling = dataset_param["validation_pct"]
    epoch_size = ( 
        int(
            dataset_param["total_data_size_per_task"] 
            * scaling 
            * len(dataset_param["tasks"]
        ) 
        * dataset_param["data_pct_usage"]) 
    )
    batch_size = (
        train_param["local_batch_size"] 
            if train_param["distributed"] is False 
            else train_param["local_batch_size"] * 1
    )
    if epoch_size % batch_size != 0:
        return epoch_size // batch_size + 1
    return epoch_size // batch_size

def get_total_batch_count(
        dataset_param: DatasetParam,
        train_param: TrainParam,
        batch_id: int, 
        epoch_id: int,
        is_train: bool = True
    ) -> int:
    batch_count_per_epoch = get_batch_per_epoch(dataset_param, train_param, is_train)
    current_total_batch_count = batch_id + epoch_id * batch_count_per_epoch
    return current_total_batch_count

def measure_lr(
        dataset_param: DatasetParam,
        train_param: TrainParam,
        batch_id: int, 
        epoch_id: int
    ):
    current_total_batch_count = get_total_batch_count(
        dataset_param, 
        train_param, 
        batch_id, 
        epoch_id, 
        is_train=True
    )
    return get_lr(current_total_batch_count)

In [ ]:
def get_lrs():
    lrs = []
    for epoch_id in range(get_train_param()["total_epoch"]):
        for batch_id in range(
            get_batch_per_epoch(
                get_dataset_param(),
                get_train_param(),
            )
        ):
            lrs.append(
                measure_lr(
                    get_dataset_param(),
                    get_train_param(),
                    batch_id,
                    epoch_id
                )
            )
    return lrs

In [ ]:
lrs = get_lrs()
print(len(lrs))
_ = plt.plot(lrs)

Eval Result

In [46]:
df = pd.read_csv('eval_2024-05-19_zesty-microwave-656_14_2024-05-23.csv')

In [47]:
task_ids = {
    "visual_manipulation": "01",
    "scene_understanding": "02",
    "rotate": "03",
    "rearrange": "04",
    "rearrange_then_restore": "05",
    "novel_adj": "06",
    "novel_noun": "07",
    "novel_adj_and_noun": "08",
    "twist": "09",
    "follow_motion": "10",
    "follow_order": "11",
    "sweep_without_exceeding": "12",
    "sweep_without_touching": "13",
    "same_texture": "14",
    "same_shape": "15",
    "manipulate_old_neighbor": "16",
    "pick_in_order_then_restore": "17"
}

In [48]:


eval_success_rates = {
    task_ids[group.iloc[0]['task']]: round(float(group[['sucess']].mean().iloc[0] * 100), 1)
        for _, group in df.groupby('task')
}

In [49]:
sorted(list(eval_success_rates.items()))

[('01', 56.0),
 ('02', 50.0),
 ('03', 29.0),
 ('04', 35.0),
 ('05', 18.0),
 ('06', 42.0),
 ('07', 60.0),
 ('09', 1.0),
 ('11', 77.0),
 ('12', 65.0),
 ('15', 49.0),
 ('16', 35.0),
 ('17', 5.0)]

In [51]:
sum(eval_success_rates.values()) / len(eval_success_rates)

40.15384615384615

In [50]:
sorted(list(eval_success_rates.items()))

[('01', 56.0),
 ('02', 50.0),
 ('03', 29.0),
 ('04', 35.0),
 ('05', 18.0),
 ('06', 42.0),
 ('07', 60.0),
 ('09', 1.0),
 ('11', 77.0),
 ('12', 65.0),
 ('15', 49.0),
 ('16', 35.0),
 ('17', 5.0)]

Evalution

In [ ]:
import pandas as pd
import glob


mode = 'train'
file_pattern = f'logs\\{mode}_*.csv'
csv_files = glob.glob(file_pattern)
dataframes = [pd.read_csv(file) for file in csv_files]
solution_df = pd.concat(dataframes, ignore_index=True)

In [ ]:
loss_cols = [col for col in df.columns if 'unweigted_sample_loss' in col]
position_loss_cols = [col for col in loss_cols if 'position' in col]
rotation_loss_cols = [col for col in loss_cols if 'rotation' in col]

In [ ]:
rename = {
    old_col_name: old_col_name.replace("unweigted_sample_loss__", "") 
        for old_col_name in loss_cols
}
df[loss_cols].rename(columns=rename).describe()

In [ ]:
for task_name, task_df in df[position_loss_cols + ['task']].groupby('task'):
    rename = {
        old_col_name: old_col_name.replace("unweigted_sample_loss__", "") 
            for old_col_name in position_loss_cols
    }
    print(task_name)
    print(task_df[position_loss_cols].rename(columns=rename).describe())
    print("=============")

In [ ]:
df['sum_loss'] = df[position_loss_cols].sum(axis=1)
df.groupby('task')['sum_loss'].describe().T.columns

In [ ]:
import pandas as pd
import glob
import os
from typing import List, Literal, Callable

In [ ]:

mode = 'train'
i = 31
    
file_pattern = f'logs/{mode}_{i}_*.csv'
csv_files = glob.glob(file_pattern)
dataframes = [pd.read_csv(file) for file in csv_files]
df = pd.concat(dataframes, ignore_index=True)

In [ ]:
loss_cols = [col for col in df.columns if 'unweigted_sample_loss' in col]
position_loss_cols = [col for col in loss_cols if 'position' in col]
rotation_loss_cols = [col for col in loss_cols if 'rotation' in col]

In [ ]:
rename = {
    old_col_name: old_col_name.replace("unweigted_sample_loss__", "") 
        for old_col_name in loss_cols
}
df[loss_cols].rename(columns=rename).describe().columns

In [ ]:
exps = set(
    [
        str(os.path.basename(file_path))
        .split('.')[0]
        .split('_')[2] 
            for file_path in glob.glob('logs/*.csv')
    ]
)

In [ ]:
def get_trace_paths(
        exp_id: str, 
        mode: str
    ):
    return [
        exp
            for exp in sorted(
                glob.glob(f'logs/{mode}_*_{exp_id}.csv'),
                key = lambda x: int(x.split('_')[1])
            )
    ]

def trace_epoch(
        exp_id: str, 
        mode: str = 'train'
    ) -> List[pd.DataFrame]:
    return [
        pd.read_csv(exp)
            for exp in 
                get_trace_paths(exp_id, mode)
    ]

def trace_ddp(exp_ids: List[str], epoch: int, mode: str = 'train') -> pd.DataFrame:
    exp_paths = [
        os.path.join('logs', f'{mode}_{epoch}_{exp_id}.csv') for exp_id in exp_ids
    ]
    exp_paths = filter(lambda x: os.path.exists(x), exp_paths)
    dataframes = [
        pd.read_csv(exp_path) for exp_path in exp_paths
    ]
    df = pd.concat(dataframes, ignore_index=True)
    return df


In [ ]:
dfs = [
    trace_ddp(exps, i, 'train') for i in range(30)
]

In [ ]:
Aggregator = Literal['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
Transform = Callable[[pd.DataFrame], pd.DataFrame]
def measure(
        df: pd.DataFrame, 
        transform: Transform, 
        attr: str, 
        aggr: Aggregator
    ) -> float:
    df = transform(df)
    describe = df.describe()
    if attr not in describe.columns:
        describe = describe.T
    return describe[attr][aggr]
    

In [ ]:
attributes = [
    'pose0_position_0', 'pose0_position_1', 'pose1_position_0',
    'pose1_position_1', 'pose0_rotation_0', 'pose0_rotation_1',
    'pose0_rotation_2', 'pose0_rotation_3', 'pose1_rotation_0',
    'pose1_rotation_1', 'pose1_rotation_2', 'pose1_rotation_3'
]
tasks = [
    'follow_order', 'manipulate_old_neighbor', 'novel_adj', 'novel_noun',
    'pick_in_order_then_restore', 'rearrange', 'rearrange_then_restore',
    'rotate', 'same_profile', 'scene_understanding', 'simple_manipulation',
    'sweep_without_exceeding', 'twist'
]

In [ ]:
def per_attr_transform(df: pd.DataFrame) -> pd.DataFrame:
    loss_cols = [col for col in df.columns if 'unweigted_sample_loss' in col]
    rename = {
        old_col_name: old_col_name.replace("unweigted_sample_loss__", "") 
            for old_col_name in loss_cols
    }
    return df[loss_cols].rename(columns=rename)

In [ ]:
measure(solution_df, per_attr_transform, 'pose0_position_0', 'mean')

In [ ]:
for attr in attributes:
    print(attr, measure(df, per_attr_transform, attr, 'mean'))

In [ ]:
def per_task_transform(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    loss_cols = [col for col in df.columns if 'unweigted_sample_loss' in col]
    df['sum_loss'] = df[loss_cols].sum(axis=1)
    return df.groupby('task')['sum_loss']

In [ ]:
for task in tasks:
    print(task, measure(
        df, 
        per_task_transform, 
        task, 
        'mean'))

In [ ]:

for task in attributes:
    print(task, measure(
        solution_df.loc[
            (solution_df['unweigted_sample_loss__pose0_position_0'] < 3.9) &
            (solution_df['unweigted_sample_loss__pose0_position_1'] < 4.6) &
            (solution_df['unweigted_sample_loss__pose1_position_0'] < 3.9) &
            (solution_df['unweigted_sample_loss__pose1_position_1'] < 4.6) &
            (solution_df['unweigted_sample_loss__pose0_rotation_0'] < 3.9) &
            (solution_df['unweigted_sample_loss__pose0_rotation_1'] < 3.9) &
            (solution_df['unweigted_sample_loss__pose0_rotation_2'] < 3.9) &
            (solution_df['unweigted_sample_loss__pose0_rotation_3'] < 3.9) &
            (solution_df['unweigted_sample_loss__pose1_rotation_0'] < 3.9) &
            (solution_df['unweigted_sample_loss__pose1_rotation_1'] < 3.9) &
            (solution_df['unweigted_sample_loss__pose1_rotation_2'] < 3.9) &
            (solution_df['unweigted_sample_loss__pose1_rotation_3'] < 3.9) 
        ], 
        per_attr_transform, 
        task, 
        'mean'))

In [ ]:
measure(solution_df, per_task_transform, 'follow_order', 'mean')

In [ ]:
follow_order = [measure(df, per_task_transform, 'follow_order', 'mean') for df in dfs]
follow_order

### Trace Accu

In [25]:
import pandas as pd
from functools import partial

In [41]:
df = pd.read_csv('eval_2024-05-19_zesty-microwave-656_14_2024-05-23.csv')

In [42]:

def per_trace_accu(row: pd.Series, col: str, threshold: float) -> float:
    correct_trace: List[int] = []
    for t_step in range(10):
        ground_truth = row.loc[f'action_trace__{t_step}__oracle_action__{col}']
        prediction = row.loc[f'action_trace__{t_step}__policy_action__{col}']
        if pd.isna(ground_truth) and pd.isna(prediction):
            break
        if pd.isna(ground_truth) or pd.isna(prediction):
            correct_trace.append(0)
            continue
        
        correct_trace.append(int(abs(ground_truth - prediction) <= threshold))
    if len(correct_trace) < 0:
        return 0
    return sum(correct_trace) / len(correct_trace)

        

In [43]:
attributes = [
    'pose0_position_0', 'pose0_position_1', 'pose1_position_0',
    'pose1_position_1', 'pose0_rotation_0', 'pose0_rotation_1',
    'pose0_rotation_2', 'pose0_rotation_3', 'pose1_rotation_0',
    'pose1_rotation_1', 'pose1_rotation_2', 'pose1_rotation_3'
]

In [52]:
accus = {
    attribute: df.apply(partial(per_trace_accu, col=attribute, threshold=0), axis=1).mean()
    for attribute in attributes 
}

In [53]:
position_accus = [accus[key] for key in accus if 'position' in key]
rotation_accus = [accus[key] for key in accus if 'position' in key]

1.3358479853479854

In [40]:
for attribute in attributes:
    print(attribute, df.apply(partial(per_trace_accu, col=attribute, threshold=2), axis=1).mean())

pose0_position_0 0.9
pose0_position_1 0.81
pose1_position_0 0.95
pose1_position_1 0.83
pose0_rotation_0 1.0
pose0_rotation_1 1.0
pose0_rotation_2 1.0
pose0_rotation_3 1.0
pose1_rotation_0 1.0
pose1_rotation_1 1.0
pose1_rotation_2 0.09
pose1_rotation_3 0.12
